<a href="https://colab.research.google.com/github/venomouscyanide/dl_sain/blob/master/week6/week6_3a_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NB: All scripts were run locally due to compute constraints faced on Colab. This notebook is maintaned only to showcase the outputs/main parts of the code. 

### Get all imports

In [ ]:
import math
import os
import string
from collections import Counter, defaultdict
from math import ceil
from typing import List, Set, Tuple, Type, Dict
from abc import abstractmethod

import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import gdown
import unidecode

### Init the class which will give the password datasets. Currently supports Mate1, 000webhost and ClixSense.

In [ ]:
class PasswordDataset:
    @abstractmethod
    def get_download_url(self) -> str:
        pass

    @abstractmethod
    def get_dataset_local_path(self) -> str:
        pass


class ClixSense(PasswordDataset):
    def get_download_url(self) -> str:
        return 'https://drive.google.com/uc?id=1S0-1gdzoP-HecS3L5_zStZhvTt9A4q97'

    def get_dataset_local_path(self) -> str:
        return 'pwd_dataset_manager/datasets/ClixSense.txt'


class WebHost(PasswordDataset):
    def get_download_url(self) -> str:
        return 'https://drive.google.com/uc?id=11tsLveuHo3xaVL2DRh3FfuUPG8LEtzYd'

    def get_dataset_local_path(self) -> str:
        return 'pwd_dataset_manager/datasets/000webhost.txt'


class Mate1(PasswordDataset):
    def get_download_url(self) -> str:
        return 'https://drive.google.com/uc?id=10LtJiV9J-Vuy1I8iSxacH4fsPqZamIeB'

    def get_dataset_local_path(self) -> str:
        return 'pwd_dataset_manager/datasets/Mate1.txt'


class DatasetFactory:
    def get(self, dataset_name: str) -> Type[PasswordDataset]:
        if dataset_name == "ClixSense":
            return ClixSense
        elif dataset_name == "000webhost":
            return WebHost
        elif dataset_name == "Mate1":
            return Mate1
        else:
            raise NotImplementedError(f"Dataset: {dataset_name} not supported")


def get_dataset(dataset_klass: Type[PasswordDataset]) -> List[str]:
    local_dataset = dataset_klass().get_dataset_local_path()
    if not os.path.exists(local_dataset):
        gdown.download(dataset_klass().get_download_url(), quiet=True, output=local_dataset)

    with open(local_dataset, "r") as dataset:
        contents = unidecode.unidecode(dataset.read())
        contents = contents.split('\n')
        contents = [content[:-1].strip() for content in contents]
    return contents

### Declare all utilities
This includes method to convert string to tensor + method for taking top 100,000 passwords from any dataset

In [ ]:
# min size of ClixSense is 6
# max size of ClixSense is 25
# most common with their no_of_occurrences
# [('123456', 17871), ('123456789', 3294), ('12345678', 2091), ('password', 1967), ('111111', 1892),
# ('1234567', 1299), ('iloveyou', 1266), ('qwerty', 1187), ('clixsense', 1172), ('000000', 977)]

def get_input_expected_clixsense(dataset: List[str]) -> Tuple[List[Tuple[str, int]], Dict[str, Set[Tuple[str, str]]]]:    
    password_slices_dict = defaultdict(set)
    # this was made to be in this way because I wanted to support more slices within a password.
    # However, this is not being done. As a result this looks really stupid.
    [password_slices_dict[pwd].add((pwd[0: -1], pwd[1:])) for pwd in dataset[:]]

    n_most_common = 100000
    all_passwords = [pwd for pwd in dataset]
    counter = Counter(all_passwords)
    most_common = counter.most_common(n_most_common)

    return most_common, password_slices_dict


def convert_str_to_tensor(string_to_convert: str):
    size = len(string_to_convert)
    converted_tensor = torch.zeros(size, 1).long()
    for index, char in enumerate(string_to_convert):
        converted_tensor[index][0] = string.printable.index(char)
    return converted_tensor.to(device)

### The GRU RNN network
This model will be trained on 100,000 most common passwords in ClixSense for a set number of epochs

In [ ]:
class GRU_RNN(nn.Module):
    def __init__(self, embedding_dim: int, hidden_size: int, no_of_hidden_layers: int, output_size: int):
        super().__init__()
        self.embedding_dim = embedding_dim
        # (L, N, H_in)
        self.gru = nn.GRU(self.embedding_dim, hidden_size, num_layers=no_of_hidden_layers)
        self.embedding = nn.Embedding(len(string.printable), embedding_dim=self.embedding_dim)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input: torch.Tensor, hidden_state: torch.Tensor):
        input = self.embedding(input)
        reshaped_input = input.view(1, 1, self.embedding_dim)
        input, hidden_state = self.gru(reshaped_input, hidden_state)
        output = self.linear(input)
        return output, hidden_state

### Init the class which will train the RNN on ClixSense
#### The rough logic for training is as follows:

for 10 epochs
>for password in 100,000 most occuring passwords in ClixSense
>>form the input at t and the expected output at t + 1 for current password `[eg; for "password", input is "passwor" and expected is "assword"]` <br>
>>for (num_of_occurrences for the current password / 100000) * 100 `[this is just to train passwords with more occurrences more times]`
>>>train the GRU on the input, expected tensors of the current password

In [ ]:
class PasswordGuesserUsingRNN:
    def __init__(self):
        self.hidden_size = 100
        self.no_of_hidden_layers = 20
        self.output_size = len(string.printable)
        self.embedding_dim = 5
        self.epochs = 10
        self.eta = 1e-4

    def train_and_evaluate(self):
        dataset_klass = DatasetFactory().get("ClixSense")
        dataset = get_dataset(dataset_klass)

        gru_model = GRU_RNN(self.embedding_dim, self.hidden_size, self.no_of_hidden_layers, self.output_size).to(device)
        optimizer = torch.optim.Adam(gru_model.parameters(), lr=self.eta)
        loss_fn = CrossEntropyLoss()

        n_most_common, pwd_inp_exp = get_input_expected_clixsense(dataset)
        # total_len of dict = 1338980
        # total length of passwords = 2221027
        for epoch in range(self.epochs):
            for pwd_index, (most_common_pwd, num_occ) in enumerate(n_most_common[:]):
                inp_target_set = pwd_inp_exp[most_common_pwd]
                for _ in range(ceil((num_occ / 100000) * 100)):
                    for input_pwd, target_pwd in inp_target_set:
                        # this loop is always only run once as per current setting
                        loss = 0
                        optimizer.zero_grad()

                        hidden_state = self._init_hidden()
                        input_tensor = convert_str_to_tensor(input_pwd)
                        target_tensor = convert_str_to_tensor(target_pwd)

                        for input, expected in zip(input_tensor, target_tensor):
                            output, hidden_state = gru_model(input, hidden_state)
                            loss += loss_fn(output[-1], expected)

                        loss.backward()
                        optimizer.step()

                        loss = loss.item() / len(input_pwd)

                if pwd_index % 1000 == 0:
                    print(f"At pwd_index: {pwd_index} of {len(n_most_common)}")
                    print(f"training password: {most_common_pwd}")

                    print(f'At epoch: {epoch} with loss: {loss}')
                    start = "123"
                    prediction = self.evaluate_password(gru_model, start, 15)
                    print(f"Prediction is {prediction} for start with '{start}'")

        return gru_model

    def evaluate_password(self, gru_model: nn.Module, password_start: str, max_length: int):
        prediction = password_start
        start_tensor = convert_str_to_tensor(password_start)
        with torch.no_grad():
            hidden_state = self._init_hidden()
            for char in start_tensor:
                _, hidden_state = gru_model(char, hidden_state)

            input = start_tensor[-1]
            for char_gen in range(max_length - len(password_start)):
                output, hidden_state = gru_model(input, hidden_state)

                # understand below; taken from the ref colab
                output_dist = output.data.view(-1).exp()
                top_i = torch.multinomial(output_dist, 1)[0]

                char_predicted = string.printable[top_i]
                prediction += char_predicted
                input = convert_str_to_tensor(char_predicted)

        return prediction

    def _init_hidden(self):
        # (D∗num_layers, N, Hout)
        return torch.zeros(self.no_of_hidden_layers, 1, self.hidden_size).to(device)

### Train the GRU model on ClixSense

In [ ]:
device = "cuda"
gru_model = PasswordGuesserUsingRNN().train_and_evaluate()
torch.save(gru_model, 'saved_gru_pwd.model')

The actual training was done locally and the logs for which can be found here: https://gist.github.com/venomouscyanide/8c4e18d042f4db891a614f838fa1b03a

The training took approximately 11 hours to complete.
After training, I saved the model for easily running experiments on the other 2 datasets.

The saved model is hosted here: https://drive.google.com/file/d/1P5_RetiDuEh-dLMpPt_sdjrSc9fX9Pej/view?usp=sharing

## Init the class that will help make password guesses

### The rough logic for creating passwords is as follows:


for each password in list of most common passwords in ClixSense
>for starting sequence taken by slicing the password with min length of 3`(eg: starting sequences of "helloworld" are ['hel', 'hell', 'hello', 'hellow', 'hellowo', 'hellowor', 'helloworl']`
>>for sequence lengths from 5 to 15
>>>for 5 guesses multiplied by "importance" factor `(guess more popular password sequences more)`
>>>>guess the password given starting sequence and the sequence length

importance factor = `ceil(no_of_occurrences / (dataset_len / importance) * 100)`
<br>`importance` variable value was set to `4` in my experiments

There is a counter kept for each guess and we break out of this segment when the counter reaches 1M guesses


### After training the stats are printed and 3 debug text files are generated. 
The debug files are as follows:
- starter passwords considered from ClixSense - `all_starters.txt`
- passwords that were not guessed - `missed_passwords.txt`
- all unique correctly guessed passwords - `unique_correct_guesses.txt`


In [ ]:
class MakePasswordGuesses:
    MIN_LENGTH: int = 5
    MAX_LENGTH: int = 15
    MAX_TRIES_PER_CONFIG: int = 5
    MAX_GUESSES: int = int(math.pow(10, 6))

    def __init__(self, model: nn.Module, verbose: bool = True):
        self.model = model
        self.verbose = verbose

    def evaluate_dataset(self, dataset_name: str) -> Tuple[Set[str], Set[str]]:
        dataset_klass = DatasetFactory().get(dataset_name)
        dataset = get_dataset(dataset_klass)

        dataset_counter = Counter(dataset)
        dataset_len = len(dataset)
        print(f'Total passwords in {dataset_name} is {dataset_len}')
        most_common = dataset_counter.most_common()

        total_correct_guesses, guessed_passwords, all_starters_used = self._make_guesses(most_common, dataset_counter,
                                                                                         dataset_len)
        missed_passwords = set(dataset_counter.keys()).difference(guessed_passwords)

        print(
            f"Unique guesses correct: {len(guessed_passwords)} and Total guesses: {total_correct_guesses} and total misses: {len(missed_passwords)}"
        )
        print(f"Coverage on {dataset_name}: {(total_correct_guesses / len(dataset)) * 100}")

        self._write_debug(dataset_name, "all_starters.txt", all_starters_used)
        self._write_debug(dataset_name, "unique_correct_guesses.txt", guessed_passwords)
        self._write_debug(dataset_name, "missed_passwords.txt", missed_passwords)

        return guessed_passwords, missed_passwords

    def _form_candidates(self, common_pwd: str) -> List[str]:
        min_len = 3
        return [common_pwd[0:end] for end in range(min_len, len(common_pwd))]

    def _make_guesses(self, most_common: List[Tuple[str, int]], dataset_counter: Counter, dataset_len: int):
        # TODO: Refactor. Super messy right now.
        importance = 4
        total_correct_guesses = 0
        uniq_guessed_passwords = set()
        all_starters_used = set()
        total_guess_tracker = 0

        seen_edge_cases = set()
        guesser = PasswordGuesserUsingRNN()
        for common_pwd, common_occ in most_common:
            starter_candidates = self._form_candidates(common_pwd)
            all_starters_used |= set(starter_candidates)

            for candidate in starter_candidates:
                if len(candidate) > self.MAX_LENGTH:
                    if candidate not in seen_edge_cases:
                        seen_edge_cases.add(candidate)
                        if total_guess_tracker > self.MAX_GUESSES:
                            return total_correct_guesses, uniq_guessed_passwords, all_starters_used

                        total_guess_tracker += 1
                        guess = candidate
                        total_correct_guesses = self._update_if_guess_is_correct(uniq_guessed_passwords, guess,
                                                                                 candidate, max_len,
                                                                                 dataset_counter, total_correct_guesses)
                    continue

                for max_len in range(self.MIN_LENGTH, self.MAX_LENGTH + 1):

                    if max_len < len(candidate):
                        continue

                    if max_len == len(candidate):
                        if candidate not in seen_edge_cases:
                            seen_edge_cases.add(candidate)
                            if total_guess_tracker > self.MAX_GUESSES:
                                return total_correct_guesses, uniq_guessed_passwords, all_starters_used

                            total_guess_tracker += 1
                            guess = candidate
                            total_correct_guesses = self._update_if_guess_is_correct(uniq_guessed_passwords, guess,
                                                                                     candidate, max_len,
                                                                                     dataset_counter,
                                                                                     total_correct_guesses)
                        continue

                    for _ in range(self.MAX_TRIES_PER_CONFIG * ceil(common_occ / (dataset_len / importance) * 100)):
                        if total_guess_tracker > self.MAX_GUESSES:
                            return total_correct_guesses, uniq_guessed_passwords, all_starters_used

                        total_guess_tracker += 1
                        if total_guess_tracker % 1000 == 0 and self.verbose:
                            print(f"At guess {total_guess_tracker} of {self.MAX_GUESSES}")

                        guess = guesser.evaluate_password(self.model, candidate, max_length=max_len)
                        total_correct_guesses = self._update_if_guess_is_correct(uniq_guessed_passwords, guess,
                                                                                 candidate, max_len,
                                                                                 dataset_counter, total_correct_guesses)

        return total_correct_guesses, uniq_guessed_passwords, all_starters_used

    def _update_if_guess_is_correct(self, uniq_guessed_passwords: Set[str], guess: str, candidate: str, max_len: int,
                                    dataset_counter: Counter, total_correct_guesses: int):
        if guess not in uniq_guessed_passwords:
            occurrences = dataset_counter.get(guess)
            if occurrences:
                if self.verbose:
                    print(
                        f"Correct guess: {guess}, for candidate: {candidate}, given max_len: {max_len} \nTotal Correct guesses so far:{total_correct_guesses}"
                    )
                total_correct_guesses += occurrences
                uniq_guessed_passwords.add(guess)
        return total_correct_guesses

    def _write_debug(self, dataset_name: str, file_name: str, data_as_set: Set[str]):
        debug_folder = f'debug_{dataset_name}'
        if not os.path.exists(debug_folder):
            os.mkdir(debug_folder)

        data_as_str = '\n'.join(data_as_set)
        with open(os.path.join(debug_folder, file_name), "w") as debug_file:
            debug_file.write(data_as_str)


In [ ]:
gru_model = torch.load('saved_gru_pwd.model')
MakePasswordGuesses(gru_model).evaluate_dataset('Mate1')

#### This was evaluated locally and the stats for this dataset are as follows:
<br>Total time taken: ~7 hours(2GB MX150 GPU, 8 core i5-8250U CPU, 16 gigs of RAM)
<br>Dataset: Mate1
<br>Total no of passwords: 27398563
<br>Total unique passwords: 9968589
<br>Unique correct guesses: 41331
<br>Total passwords guesses(includes duplicates): 4447756
<br>Unique passwords misses(not guessed): 9927258 
<br>Accuracy:  (4447756/27398563) * 100 =  **16.23353750340848%** 
<br>Link to guessing logs: https://gist.githubusercontent.com/venomouscyanide/a69beec369b3b15a2a63f1b98e3bfd90/raw/47901f126aeb61cba98e3424c662e6d40e4dc550/guessing_logs_mate1.log
<br>Link to starter passwords text file: https://drive.google.com/file/d/1ddsIKMiagCdmDPqvbvGRAYdUX99BL-Xx/view?usp=sharing
<br>Link to missed passwords text file: https://drive.google.com/file/d/17EfK3R0C4uFVGxLbbnKUPNhib-Kn5gc3/view?usp=sharing
<br>Link to unique guesses passwords text file: https://drive.google.com/file/d/1CUPu8RR_rIaT9h0sU-jIwq_sfNheAm1q/view?usp=sharing

In [ ]:
gru_model = torch.load('saved_gru_pwd.model')
MakePasswordGuesses(gru_model).evaluate_dataset('000webhost')

#### The performance on 000webhost was rather poor. The stats are as follows:
<br>Total time taken: ~7 hours(2GB MX150 GPU, 8 core i5-8250U CPU, 16 gigs of RAM)
<br>Dataset: 000webhost
<br>Total no of passwords: 14939552
<br>Total unique passwords: 9805274
<br>Unique correct guesses: 17591
<br>Total passwords guesses(includes duplicates): 836296
<br>Unique passwords misses(not guessed): 9787683
<br>Accuracy:  (836296/14939552) * 100 = **5.597865317514207%**
<br>Link to guessing logs: https://gist.githubusercontent.com/venomouscyanide/3c1b4837c2b462ba40fb4b08e27ebeb4/raw/a796f2ee3147fc1a3994c1ce7c5f79b29ac4bbbe/guessing_logs_000webhost.log
<br>Link to starter passwords text file: https://drive.google.com/file/d/1QXFjM9whNOqPLxdwV7uJLDKK2XREvj8f/view?usp=sharing
<br>Link to missed passwords text file: https://drive.google.com/file/d/1GsrBvZT9pYrIth2guSCV8yG6MroEJhbm/view?usp=sharing
<br>Link to unique guesses passwords text file: https://drive.google.com/file/d/12VJV4Y0dWhUfrKNzJIf0U2i728sP-UJv/view?usp=sharing